# Ejercicio resuelto: Matriz de confusión con *iris.csv*

**Objetivo**: cargar el dataset `iris.csv`, entrenar un modelo sencillo y obtener la matriz de confusión y las métricas:
- Exactitud (Accuracy)
- Precisión (Precision)
- Sensibilidad / TPR (Recall)
- Especificidad / TNR (Specificity)
- Razón de falsos positivos (FPR)

Clasificación binaria: **versicolor** (1) vs. **resto** (0).

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import datasets

def load_iris_from_csv_or_sklearn(url: str):
    try:
        df = pd.read_csv(url)
        if 'species' in df.columns:
            return df
    except Exception:
        pass
    # Fallback: sklearn iris
    iris = datasets.load_iris(as_frame=True)
    df = iris.frame.copy()
    rename_map = {
        'sepal length (cm)': 'sepal_length',
        'sepal width (cm)': 'sepal_width',
        'petal length (cm)': 'petal_length',
        'petal width (cm)': 'petal_width',
    }
    df = df.rename(columns=rename_map)
    species_map = {0: 'setosa', 1: 'versicolor', 2: 'virginica'}
    df['species'] = df['target'].map(species_map)
    return df[['sepal_length','sepal_width','petal_length','petal_width','species']]

## 1) Cargar datos y preparar el problema (versicolor vs resto)

In [ ]:
url = 'https://raw.githubusercontent.com/mwaskom/seaborn-data/master/iris.csv'
df = load_iris_from_csv_or_sklearn(url)
df['is_versicolor'] = (df['species'] == 'versicolor').astype(int)
X = df[['sepal_length','sepal_width','petal_length','petal_width']].values
y = df['is_versicolor'].values
RANDOM_STATE = 42
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=RANDOM_STATE, stratify=y
)
df.head()

## 2) Entrenar un modelo sencillo

In [ ]:
clf = LogisticRegression(max_iter=1000, solver='liblinear', random_state=RANDOM_STATE)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_pred[:10]

print(clf)

## 3) Matriz de confusión

In [ ]:
cm = confusion_matrix(y_test, y_pred, labels=[0,1])
TN, FP, FN, TP = cm.ravel()
cm

In [ ]:
from matplotlib.colors import LinearSegmentedColormap

brown_to_blue = LinearSegmentedColormap.from_list(
    "brown_to_blue",
    [
        (0.00, "#F4E1C6"),  # marrón muy claro (arena)
        (0.35, "#CFA77B"),  # marrón/tan medio
        (0.60, "#9BA8B3"),  # puente neutro (gris-azulado suave)
        (0.80, "#3C6EA6"),  # azul medio
        (1.00, "#0B3D91"),  # azul oscuro
    ],
    N=256
)

fig, ax = plt.subplots(figsize=(5,4))
im = ax.imshow(cm, interpolation='nearest', cmap=brown_to_blue)
plt.title('Confusion matrix')
plt.colorbar(im, ax=ax)
tick_marks = np.arange(2)
ax.set_xticks(tick_marks); ax.set_xticklabels([0,1])
ax.set_yticks(tick_marks); ax.set_yticklabels([0,1])
ax.set_xlabel('Predicted label'); ax.set_ylabel('Actual label')
thresh = cm.max()/2.0
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, f"{cm[i,j]}", ha='center', va='center',
                color='white' if cm[i,j] > thresh else 'black')
fig.tight_layout(); plt.show()

## 4) Cálculo de métricas

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report
def safe_div(num, den):
    return float(num)/den if den!=0 else float('nan')

#accuracy = 
#precision = 
#recall =          # Sensibilidad / TPR
#specificity =    # Especificidad / TNR
#fpr =             # 1 - Specificity

print('TP, TN, FP, FN =', TP, TN, FP, FN)
print(f'Accuracy   : {accuracy:.3f}')
print(f'Precision  : {precision:.3f}')
print(f'Recall/TPR : {recall:.3f}')
print(f'Specificity: {specificity:.3f}')
print(f'FPR        : {fpr:.3f}')


## 5) Resultados (con `random_state=42`)
- **TP**: 3
- **TN**: 25
- **FP**: 5
- **FN**: 12
- **Accuracy**: 0.622
- **Precision**: 0.375
- **Recall/TPR (Sensibilidad)**: 0.200
- **Specificity/TNR (Especificidad)**: 0.833
- **FPR (1 - Specificity)**: 0.167

> Nota: si cambias el `random_state` o el modelo, los valores podrán variar.